In [1]:
using Laplacians

objc[16736]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16736]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16736]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[16736]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [12]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [13]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [14]:
graphp = "../graphs/xu_3/"

"../graphs/xu_3/"

In [15]:
a = readFromFile(graphp * "graph.mtx");

In [16]:
tree = readFromFile(graphp * "tree.mtx");

In [17]:
x = readFromFile(graphp * "x.vec");

In [18]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);
a.n

9261

In [19]:
@time F = treeSolver(tree)
@time F(b);

  0.035625 seconds (8.14 k allocations: 3.067 MB)
  0.055295 seconds (160.20 k allocations: 3.441 MB, 8.41% gc time)


In [21]:
# a run on standard Float64 data types
numIts = 5000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 320.7116177504572
Working on iteration 20 with a-norm error 312.67845868247554
Working on iteration 30 with a-norm error 301.1536938840592
Working on iteration 40 with a-norm error 283.8507667615667
Working on iteration 50 with a-norm error 271.95196671993466
Working on iteration 60 with a-norm error 256.00923558018053
Working on iteration 70 with a-norm error 242.47236035535335
Working on iteration 80 with a-norm error 229.45448563064681
Working on iteration 90 with a-norm error 217.23323051587857
Working on iteration 100 with a-norm error 199.55033283597973
Working on iteration 110 with a-norm error 185.8199230436832
Working on iteration 120 with a-norm error 175.84385062753444
Working on iteration 130 with a-norm error 162.75357412975103
Working on iteration 140 with a-norm error 150.91940387368703
Working on iteration 150 with a-norm error 139.42281127745724
Working on iteration 160 with a-norm error 130.93057222575723
Working on iteration 

In [22]:
numIts = 5000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 320.71161775045715
Working on iteration 20 with a-norm error 312.6784586824755
Working on iteration 30 with a-norm error 301.15369388380026
Working on iteration 40 with a-norm error 280.8917444240631
Working on iteration 50 with a-norm error 268.11438466108973
Working on iteration 60 with a-norm error 253.0705810173173
Working on iteration 70 with a-norm error 239.0555511850014
Working on iteration 80 with a-norm error 223.92403144626178
Working on iteration 90 with a-norm error 208.28022064333499
Working on iteration 100 with a-norm error 193.99518106536755
Working on iteration 110 with a-norm error 177.06233936671856
Working on iteration 120 with a-norm error 167.15394799192043
Working on iteration 130 with a-norm error 153.68985040584695
Working on iteration 140 with a-norm error 139.5671162020303
Working on iteration 150 with a-norm error 129.18228144926158
Working on iteration 160 with a-norm error 121.66651462686711
Working on iteration 1